Script to translate README.md relative urls

Creates output/README.md for usage with pypi

Requires toml !

In [30]:

import re
import toml
import argparse
import posixpath

from pathlib import Path



In [31]:

PYPROJECT = Path.cwd().joinpath("../pyproject.toml").resolve(strict=True)
ROOTDIR = PYPROJECT.parent


In [32]:

def jquery(data: dict, item: str, default=None):
    result = data

    for i in item.split("."):
        result = result.get(i, None)
        if result is None:
            return default

    return result


def get_project_url(pyproject=PYPROJECT):
    """extract project url from project configuration"""

    if not pyproject.exists():
        raise FileNotFoundError("pyproject.toml")

    config = toml.load(pyproject)
    
    return jquery(config, "project.urls.homepage")

get_project_url()



'https://github.com/furechan/mintalib'

In [33]:
def process_readme(file, *, project_url=None, branch="main", verbose=True):
    """translate relative urls to full urls"""

    if project_url is None:
        project_url = get_project_url()

    def replace(m):
        exclam, alt, url = m.groups()
        ftype = "raw" if exclam else "blob"
        if re.fullmatch("[^:/][^:]*", url):
            url = posixpath.join(project_url, ftype, branch, url)
            text = f"{exclam}[{alt}]({url})"
            if verbose:
                print("mapping", m.group(0), "->", text)
        else:
            text = m.group(0)
        return text

    source = file.read_text()

    result = re.sub(
        r"(\!?) \[ ([^]]*) \] \( ([^)]+) \)", replace, source,
        flags = re.VERBOSE
        )

    return result

README = ROOTDIR.joinpath("README.md").resolve(strict=True)

output = process_readme(README, verbose=True)



mapping [mintalib.core](docs/mintalib.core.md) -> [mintalib.core](https://github.com/furechan/mintalib/blob/main/docs/mintalib.core.md)
mapping [mintalib.functions](docs/mintalib.functions.md) -> [mintalib.functions](https://github.com/furechan/mintalib/blob/main/docs/mintalib.functions.md)
mapping [mintalib.indicators](docs/mintalib.indicators.md) -> [mintalib.indicators](https://github.com/furechan/mintalib/blob/main/docs/mintalib.indicators.md)


In [34]:
OUTDIR = ROOTDIR / "output"
OUTDIR.mkdir(exist_ok=True)


output = process_readme(README, verbose=True)
outfile = OUTDIR.joinpath("README.md").resolve()

outfile.write_text(output)

mapping [mintalib.core](docs/mintalib.core.md) -> [mintalib.core](https://github.com/furechan/mintalib/blob/main/docs/mintalib.core.md)
mapping [mintalib.functions](docs/mintalib.functions.md) -> [mintalib.functions](https://github.com/furechan/mintalib/blob/main/docs/mintalib.functions.md)
mapping [mintalib.indicators](docs/mintalib.indicators.md) -> [mintalib.indicators](https://github.com/furechan/mintalib/blob/main/docs/mintalib.indicators.md)


8734

In [35]:
import re 

re.fullmatch("[^:/][^:]*", "aaa/bbb.cc")

<re.Match object; span=(0, 10), match='aaa/bbb.cc'>

In [36]:
posixpath.join("http://acme.com", "aaa", "bbb/toto.md")

'http://acme.com/aaa/bbb/toto.md'